# Federated learning for the clinical classification of genetic variants

In this notebook we will build a federated learning system for the clinical annotation of genetic variants in a multi-site clinical setting.

**What are genetic variants ?**

Genetic variants are differences in the DNA sequence among individuals. These variations can occur at a single position in the genome (called a single nucleotide polymorphism, or SNP), or involve larger segments of DNA, such as insertions, deletions, or duplications.

They play a crucial role in:

- Determining traits like eye color or blood type.
- Influencing the risk of developing certain diseases.
- Affecting how individuals respond to medications.

Variants can be benign, pathogenic, or of unknown significance, depending on their impact on health and biological function.

![Genetic Variants](https://d7xvj3qn4ee4u.cloudfront.net/wp-content/uploads/2024/09/09014055/Screenshot-2024-10-08-at-4.40.25%E2%80%AFPM.png)

**What is federated learning ?**

Federated Learning is a machine learning technique that allows multiple devices or servers to collaboratively train a model without sharing their raw data. Instead of sending data to a central server, each device trains the model locally and only shares model updates (like weights or gradients) with a central server.

**Federated Training: Step-by-Step**

Federated learning typically proceeds in rounds. Here's what happens in each round:

Step 1: Initialization

- A central server creates a global model (initial weights).
- The model is sent to a group of participating clients (e.g., hospital, research institutions, clinical testing laboratories).

Step 2: Client Selection

- The server selects a subset of clients for this round.
- Selection can be random or based on availability, bandwidth, or past participation.

Step 3: Local Training

- Each selected client receives the current global model.
- The client trains this model on its own local dataset (which never leaves the device).
- After local training, each client computes model updates (e.g., changes to weights or gradients).

Step 4: Upload Updates

- Clients send their model updates back to the server.
- Only the updates are transmitted — not the raw data.

Step 5: Aggregation

- The server aggregates the updates from all clients (typically using a method like Federated Averaging).
- This means averaging the weights/gradients from each client, weighted by the size of their datasets.

Step 6: Model Update

- The aggregated result is used to update the global model.
- This new global model ideally performs better, having learned from all clients' local data.

Step 7: Repeat

- Steps 2–6 are repeated for multiple rounds until the global model converges (performance stabilizes or meets a target).


# Implementing the simulation

Concretely, we will simulate a federated learning scenario involving three clients collaborating to train a machine learning model. The objective is to classify genetic variants as either pathogenic or non-pathogenic. To build and simulate this setup, we will use PyTorch for model development and Flower (FL) for orchestrating the federated learning process.

Let's start by installing the required libraries:

In [1]:
!pip install flwr[simulation]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.3
    Uninstalling typer-0.15

In [ ]:
#!pip install -q torch

In [ ]:
#!pip install pandas

In [ ]:
#!pip install numpy

Now that we have all the dependencies, let's import everything we need for the simulation.

In [2]:
#importing all the required libraries

from flwr.common import NDArrays, Scalar, Context, Parameters, Metrics, ndarrays_to_parameters
from flwr.client import NumPyClient, ClientApp
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from collections import OrderedDict
from typing import List, Tuple
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import json
import torch
import numpy as np
import pandas as pd

# Defining the toy dataset

For this simulation, we will create a toy dataset in which each genetic variant is represented by a set of numerical features. Additionally, each variant will be associated with the chromosome it resides on and the client that holds it, simulating distributed ownership of the data.

In [3]:
#creating the dataset for the experiments

def generate_dataset(no_clients, size_clients, no_features, random_state=123):
  """
  Generates a dataframe with the following columns:
  - client : to identify the client in the federations
  - chromosome: to identify the chromosome where the variant is located
  - feature_1, ... feature_n: numerical values scaled between -1 and 1
  - label : the label of the variant (0 or 1)
  The client i holds a local dataset of size size_clients[i]
  """

  # Set the seed for reproducibility
  rng = np.random.default_rng(seed=random_state)

  #generating a matrix of shape size_clients x no_features
  samples = np.random.uniform(low=-1.0, high=1.0, size=(np.sum(size_clients),no_features))

  #generating an array of len np.sum(size_clients) with numbers between 1 and 23 (representing the chromosomes)
  chromosomes_ = np.random.randint(low=1, high=24, size=np.sum(size_clients))

  #generating the labels
  labels = np.random.randint(low=0, high=2, size=np.sum(size_clients))

  #assigning the clients to each sample
  clients = []
  for i in range(no_clients):
    clients.extend([f"client {i+1}"]*size_clients[i])

  #creating the dataframe
  df = pd.DataFrame(samples, columns=[f"feature_{i+1}" for i in range(no_features)])
  df["client"] = clients
  df["chromosome"] = chromosomes_
  df["label"] = labels

  return df


Now, we can call the function and visualise our toy dataset

In [68]:
df_all_clients = generate_dataset(3,[50,60,30],10)
df_all_clients

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,client,chromosome,label
0,-0.289593,-0.706447,-0.992226,0.006872,-0.498566,0.765898,-0.218068,0.490813,-0.052231,0.274712,client 1,15,0
1,0.076661,-0.220545,-0.438328,-0.721690,0.025372,0.298995,0.644204,0.063840,-0.775390,-0.021358,client 1,22,0
2,0.743352,0.801106,0.063259,-0.978706,0.566521,-0.134671,-0.123946,-0.578891,0.562039,0.669613,client 1,5,1
3,-0.815443,0.496108,0.020322,-0.577497,0.172636,0.276130,0.931906,0.323648,-0.652478,-0.835909,client 1,9,1
4,-0.856792,-0.387612,0.928111,0.985511,0.815216,-0.376771,-0.996408,-0.049297,0.886050,0.449671,client 1,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0.167131,0.126068,-0.234959,0.663575,-0.004774,-0.184335,-0.664814,-0.871065,0.196466,0.789899,client 3,11,0
136,-0.979946,-0.059245,-0.429065,-0.339708,0.848948,-0.492547,-0.422696,-0.535086,0.303652,-0.835814,client 3,15,0
137,-0.149506,0.392713,-0.108300,-0.796498,0.859313,-0.797899,-0.660769,0.442901,0.676182,0.906794,client 3,7,1
138,0.518441,-0.966502,-0.320052,-0.914001,0.490947,0.532773,-0.877111,-0.770301,-0.454171,0.241028,client 3,1,1


# Defining the machine learning model

Let's consider a simple Multilayer Perceptron model with just one hidden layer.

In [5]:

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out


Let's define the functions for training and evaluating the MLP model.

In [6]:
def train(model,train_loader, optimizer, train_params):

  epochs = train_params["epochs"]
  learning_rate = train_params["learning_rate"]
  device = torch.device(train_params["device"])

  model.to(device)
  model.train()
  criterion = nn.CrossEntropyLoss()
  avg_loss = []


  for epoch in range(epochs):

    for batch_data in train_loader:
      data, target = batch_data[0].to(device), batch_data[1].to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output,target.long())
      loss.backward()
      optimizer.step()
      avg_loss.append(loss.item())

    print(f"Avg training loss = {np.mean(avg_loss)}")



def test(model,test_loader,test_params):

  device = test_params["device"]
  accuracy, loss_ = 0,0
  criterion = nn.CrossEntropyLoss()

  model.to(device)
  model.eval()

  with torch.no_grad():

    for batch_data in test_loader: #here we just consider one batch
      data, target = batch_data[0].to(device), batch_data[1].to(device)
      output = model(data)
      loss_ = criterion(output,target.long())
      pred = torch.max(output.data.cpu(), 1)[1]

      accuracy = np.sum(pred.numpy() == target.numpy()) / len(target.numpy())


  return loss_, accuracy

Now, let's define a function for training the MLP model for a number of epochs and evaluate it on an independent test set.

In [7]:
def run_centralized(model, train_loader, test_loader, train_params, test_params):

  epochs = train_params["epochs"]

  #changing epochs to 1 for commodity
  train_params["epochs"] = 1

  #defining the optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=train_params["learning_rate"])

  for i in range(epochs):
    print(f"Training epoch {i}")
    train(model, train_loader, optimizer, train_params)

  #evaluating the model
  test_loss, test_accuracy = test(model, test_loader, test_params)
  print(f"Test accuracy: {test_accuracy}")

  return test_loss, test_accuracy


Now we will train a model to classify whether a genetic variant located on chromosome 1 is pathogenic or non-pathogenic. To avoid data contamination and ensure a clear separation between training and testing data, the model will be trained exclusively on variants from chromosomes 2, 3, ..., X, and evaluated solely on variants from chromosome 1. This setup simulates a real-world generalization task across genomic regions.

In [8]:
def get_features_labels(df):
  # drop columns client and chromosome
  y = df["label"]
  X = df.drop(["label", "client", "chromosome"], axis=1)
  X = X.to_numpy(dtype = "float32")
  y = np.array(y)
  return X, y

In [72]:
train_params = {"epochs": 100, "learning_rate": 0.1, "device": "cpu"}
test_params = {"device": "cpu"}
batch_size = 8

#creating the train_loader and test_loader
train_df = df_all_clients[df_all_clients["chromosome"] != 1]
test_df = df_all_clients[df_all_clients["chromosome"] == 1]

train_X, train_y = get_features_labels(train_df)
val_X, val_y = get_features_labels(test_df)

train_dataset = TensorDataset(torch.Tensor(train_X), torch.Tensor(train_y))
val_dataset = TensorDataset(torch.Tensor(val_X), torch.Tensor(val_y))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_y), shuffle=False)

#creating the MLP model
input_size = 10
hidden_size = 3
num_classes = 2

model = Net(input_size, hidden_size, num_classes)

#training
test_loss, test_accuracy = run_centralized(model, train_loader, val_loader, train_params, test_params)

Training epoch 0
Avg training loss = 0.7182602286338806
Training epoch 1
Avg training loss = 0.725940862122704
Training epoch 2
Avg training loss = 0.6994296943440157
Training epoch 3
Avg training loss = 0.6943936207715202
Training epoch 4
Avg training loss = 0.671909560175503
Training epoch 5
Avg training loss = 0.6740736645810744
Training epoch 6
Avg training loss = 0.7024425653850331
Training epoch 7
Avg training loss = 0.6845534303609062
Training epoch 8
Avg training loss = 0.6636651088209713
Training epoch 9
Avg training loss = 0.6584507928175085
Training epoch 10
Avg training loss = 0.6430139103356529
Training epoch 11
Avg training loss = 0.6215745035339805
Training epoch 12
Avg training loss = 0.6457350622205174
Training epoch 13
Avg training loss = 0.6484303439364714
Training epoch 14
Avg training loss = 0.6433305810479557
Training epoch 15
Avg training loss = 0.6301255331319922
Training epoch 16
Avg training loss = 0.6169614336069893
Training epoch 17
Avg training loss = 0.620

After 100 epochs, we observe a decrease in training loss, indicating that the model is learning effectively. Improved accuracy may be achieved through hyperparameter tuning, such as adjusting the learning rate, batch size, or model architecture.

# Federated Learning

To simulate a federated learning setup, we need to define two main components: the client and the server.

- The client is responsible for local training and evaluation, as well as for exchanging model parameters with the server. Each client operates on its own local dataset and contributes to the global model without sharing raw data.
- The server handles the aggregation of local model updates from the clients and optionally the aggregation of evaluation metrics. It coordinates the training rounds and updates the global model based on the clients’ contributions.

Let's start by defining the client:

In [10]:

class CustomClient(NumPyClient):

  def __init__(self, model, train_loader, val_loader, params_training, params_testing):

    super().__init__()
    self.model = model
    self.train_loader = train_loader
    self.val_loader = val_loader
    self.params_training = params_training
    self.params_testing = params_testing


  def fit(self, params, config):
    #copying the parameters of the global model sent by the server
    self.update_model_params(params)

    #setting the optimizer
    optimizer = self.set_optimizer()

    # local training
    train(self.model, self.train_loader, optimizer, self.params_training)

    #returning the parameters of the updated model and the size of the training set
    return get_params(self.model), len(self.train_loader), {}


  def evaluate(self, params, config):

    #copying the parameters of the global model into the local model
    self.update_model_params(params)

    #local evaluation
    loss, accuracy = test(self.model, self.val_loader, self.params_testing)


    return float(loss), len(self.val_loader), {"accuracy": float(accuracy)}

  def update_model_params(self, params):
    params_dict = zip(self.model.state_dict().keys(), params)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    self.model.load_state_dict(state_dict, strict=True)

  def set_optimizer(self):
    return torch.optim.SGD(self.model.parameters(), lr=self.params_training["learning_rate"], momentum=self.params_training["momentum"])

def get_params(model):
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

Now, let's define the aggregation at server side.

Since we are using Flower's simulation module, let's define a function to partition and assign local datasets to each client in our federated learning setup. This function will simulate the distribution of data across clients, mimicking a decentralized environment where each client holds its own subset of the dataset.

In [13]:
def client_partition(context):

  """ Returns a CustomClient where train_loader is the variants located in all chromosomes,
  except the given chromosome; and val_loader is the variants located in the given chromosome.
  """
  partition_id = int(context.node_config["partition-id"]) #this is the id of the client

  client_df = df_all_clients[df_all_clients["client"] == f"client {partition_id + 1}"] #the variants belonging to the given client

  train_df = client_df[client_df["chromosome"] != chromosome]
  val_df = client_df[client_df["chromosome"] == chromosome]

  train_X, train_y = get_features_labels(train_df)
  val_X, val_y = get_features_labels(val_df)

  train_dataset = TensorDataset(torch.Tensor(train_X), torch.Tensor(train_y))
  val_dataset = TensorDataset(torch.Tensor(val_X), torch.Tensor(val_y))

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=len(val_y), shuffle=False)

  model = Net(input_size, hidden_size, num_classes)

  client = CustomClient(model, train_loader, val_loader, train_params, test_params).to_client()
  return client


Let's define the server.

In [14]:

# Define metric aggregation function
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [38]:

def server_fn(context):

  #creating the model
  model = Net(input_size, hidden_size, num_classes)
  params = get_params(model)

  global_model_init = ndarrays_to_parameters(params)

  # Define the strategy
  strategy = FedAvg(
      fraction_fit=0.5,  # 50% clients sampled each round to do fit()
      fraction_evaluate=1.0,  # 100% clients sample each round to do evaluate()
      evaluate_metrics_aggregation_fn=weighted_average,  # callback defined earlier
      initial_parameters=global_model_init,  # initialised global model
  )

  # Construct ServerConfig
  config = ServerConfig(num_rounds=num_rounds)

  # Wrap everything into a `ServerAppComponents` object
  return ServerAppComponents(strategy=strategy, config=config)


Now, we are ready to launch the federated learning simulation

In [69]:

#global parameters

#params related to the NN
input_size = 10
hidden_size = 3
num_classes = 2

#params related to training and evaluation
batch_size = 8
train_params = {"epochs":10, "learning_rate": 0.001, "device": "cpu", "momentum":0.9}
test_params = {"device": "cpu"}

#params related to federated training
NUM_PARTITIONS = 3
num_rounds = 30

#dataset
#df_all_clients = generate_dataset(3,[50,60,30],10)

chromosomes = [1]
chromosome = None

for chr in chromosomes:

  chromosome = chr

  print(f"Training model chromosome {chromosome} ...")

  #creating the client_app
  client_app = ClientApp(client_fn=client_partition)

  #creting the server_app
  server_app = ServerApp(server_fn=server_fn)

  run_simulation(
      server_app=server_app, client_app=client_app, num_supernodes=NUM_PARTITIONS
  )

  #print(f"Training model chromosome {chromosome} completed")

INFO :      Starting Flower ServerApp, config: num_rounds=30, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 3)


Training model chromosome 1 ...


(pid=30402) 2025-05-07 13:46:02.976488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=30402) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=30402) E0000 00:00:1746625563.016447   30402 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=30402) E0000 00:00:1746625563.027911   30402 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=30402) Avg training loss = 0.7113528102636337
(ClientAppActor pid=30402) Avg training loss = 0.7086473815143108
(ClientAppActor pid=30402) Avg training loss = 0.7078515415390333
(ClientAppActor pid=30402) Avg training loss = 0.7023387625813484
(ClientAppActor pid=30402) Avg training loss = 0.7000247925519943
(ClientAppActor pid=30402) Avg training loss = 0.6987776632110277
(ClientAppActor pid=30402) Avg training loss = 0.6978964390499252
(ClientAppActor pid=30402) Avg training loss = 0.6967508913949132
(ClientAppActor pid=30402) Avg training loss = 0.6959417528576322
(ClientAppActor pid=30402) Avg training loss = 0.6946498461067676
(ClientAppActor pid=30402) Avg training loss = 0.6740006506443024
(ClientAppActor pid=30402) Avg training loss = 0.6751462146639824
(ClientAppActor pid=30402) Avg training loss = 0.67508764564991
(ClientAppActor pid=30402) Avg training loss = 0.6759313978254795
(ClientAppActor pid=30402) Avg training loss = 0.67467220723629
(ClientAppActo

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 2 clients (out of 3)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


...

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 30]
INFO :      configure_fit: strategy sampled 2 clients (out of 3)
INFO :      aggregate_fit: received 2 results and 0 failures


(ClientAppActor pid=30402) Avg training loss = 0.6915487945079803
(ClientAppActor pid=30402) Avg training loss = 0.6944603075583776
(ClientAppActor pid=30402) Avg training loss = 0.6952521072493659
(ClientAppActor pid=30402) Avg training loss = 0.6944151297211647
(ClientAppActor pid=30402) Avg training loss = 0.6950712621212005
(ClientAppActor pid=30402) Avg training loss = 0.6951286345720291
(ClientAppActor pid=30402) Avg training loss = 0.6942006193456196
(ClientAppActor pid=30402) Avg training loss = 0.6947067677974701
(ClientAppActor pid=30402) Avg training loss = 0.6945331659581926
(ClientAppActor pid=30402) Avg training loss = 0.6940199712912242
(ClientAppActor pid=30402) Avg training loss = 0.6654658168554306
(ClientAppActor pid=30402) Avg training loss = 0.6528930198401213
(ClientAppActor pid=30402) Avg training loss = 0.6531965099275112
(ClientAppActor pid=30402) Avg training loss = 0.6519958721473813


INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=30402) Avg training loss = 0.6561508394777775
(ClientAppActor pid=30402) Avg training loss = 0.654998284454147
(ClientAppActor pid=30402) Avg training loss = 0.6571273511009557
(ClientAppActor pid=30402) Avg training loss = 0.6578855277039111
(ClientAppActor pid=30402) Avg training loss = 0.6552431984908051
(ClientAppActor pid=30402) Avg training loss = 0.6539468739181757


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 30 round(s) in 33.21s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.6903103788693746
INFO :      		round 2: 0.6722917159398397
INFO :      		round 3: 0.6752743124961853
INFO :      		round 4: 0.6597104072570801
INFO :      		round 5: 0.6534581979115804
INFO :      		round 6: 0.6484582622845968
INFO :      		round 7: 0.6440479755401611
INFO :      		round 8: 0.652410606543223
INFO :      		round 9: 0.6592251062393188
INFO :      		round 10: 0.6490387717882792
INFO :      		round 11: 0.6489945451418558
INFO :      		round 12: 0.6454336444536845
INFO :      		round 13: 0.6473626693089803
INFO :      		round 14: 0.6561552484830221
INFO :      		round 15: 0.6627985636393229
INFO :      		round 16: 0.6535506645838419
INFO :      		round 17: 0.6614453196525574
INFO :      		round 18: 0.6674445271492004
INFO :      		round 19: 0.6733475128809611
INF

This example uses the Federated Averaging (FedAvg) algorithm for aggregating model updates across clients. Note that due to the random selection of clients in each training round, results may vary between runs. As with centralized machine learning, hyperparameter tuning is essential to optimize model performance and ensure reliable results.